In [2]:
import pandas as pd
import numpy as np
import torch
import ee
import sys

sys.path.append('../../')

from src.data.ee_utils import *
from src.data.data_utils import *
from src.data.dataset import *

ee.Initialize()
ee.Authenticate()


/u/ayang1/.conda/envs/CropTS/lib/python3.11/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


True

In [10]:
GAUL1 = ee.FeatureCollection('FAO/GAUL/2015/level1')
GAUL0 = ee.FeatureCollection('FAO/GAUL/2015/level0')

country = GAUL0.filter(ee.Filter.eq('ADM0_NAME', 'Bulgaria'))
provinces = GAUL1.filter(ee.Filter.eq('ADM0_NAME', 'Germany'))
provinces = provinces.toList(provinces.size().getInfo())
subset1_geom = ee.FeatureCollection(provinces.slice(0, int(0.5 * provinces.size().getInfo())))
subset2_geom = ee.FeatureCollection(provinces.slice(int(0.5 * provinces.size().getInfo()), provinces.size().getInfo()))

In [11]:
Map = geemap.Map()
Map.addLayer(country)
Map.centerObject(country)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [12]:
roi = country
start_date = '2018-01-01'
end_date = '2019-01-01'
region = 'Bulgaria'
year = '2018'
step = 10 #Days to step for averages
export_scale = 10

In [13]:
data, labels, ee_asset, fused_df = generate_fused(roi.geometry(), start_date, end_date, step, export_scale=30)

Loading files
Creating dataset of size 6287


100%|██████████| 6287/6287 [00:02<00:00, 2987.28it/s]


In [19]:
csv_save_path = '/scratch/bbug/ayang1/raw_data/lucas/fused_lucas_2018'
fused_df.to_csv(os.path.join(csv_save_path, f'multiband_point_10days_10m_1Jan-31Dec_{region}1.csv'))

In [15]:
np.save(f'/scratch/bbug/ayang1/datasets/lucas_fused/raw/{region}_data.npy', data, allow_pickle=True)
np.save(f'/scratch/bbug/ayang1/datasets/lucas_fused/raw/{region}_labels.npy', labels, allow_pickle=True)

In [16]:
Map.addLayer(ee_asset)
Map

In [17]:
data.shape

(1638, 8, 36)

In [21]:
v1 = pd.read_csv(os.path.join(csv_save_path, f'multiband_point_10days_10m_1Jan-31Dec_{region}1.csv'))
v2 = pd.read_csv(os.path.join(csv_save_path, f'multiband_point_10days_10m_1Jan-31Dec_{region}.csv'))

In [23]:
v1

,Unnamed: 0,0_B1_mean_post_20180101,0_B2_mean_post_20180101,0_B3_mean_post_20180101,0_B4_mean_post_20180101,0_B5_mean_post_20180101,0_B7_mean_post_20180101,0_VH_20180101,0_VV_20180101,10_B1_mean_post_20180411,...,8_VV_20180322,9_B1_mean_post_20180401,9_B2_mean_post_20180401,9_B3_mean_post_20180401,9_B4_mean_post_20180401,9_B5_mean_post_20180401,9_B7_mean_post_20180401,9_VH_20180401,9_VV_20180401,LABEL
0,34,447,600,708,1374,2000,1183,-22.702483,-12.984353,562.677429,...,-9.469283,546.870972,729.516113,829.161316,1760.483887,1976.774170,1436.838745,-22.295915,-7.278110,B31
1,45,446,540,631,1396,1405,908,-20.375984,-11.992291,566.774170,...,-13.319431,568.709656,837.290344,950.096802,2251.032227,2066.580566,1601.677368,-20.624519,-13.836240,B16
2,46,612,732,878,1488,1320,922,-21.093550,-11.249950,626.870972,...,-13.078944,626.548401,911.967712,1078.580688,2212.580566,2098.741943,1665.387085,-21.215304,-12.756483,B16
3,47,412,508,611,1305,1277,913,-21.735537,-13.037493,577.645142,...,-10.814573,569.258057,847.838684,1070.516113,2143.709717,2071.387207,1694.838745,-18.662707,-11.318751,B16
4,48,439,504,639,1258,1218,887,-20.146213,-13.787841,574.935486,...,-10.513015,569.774170,866.419373,1085.483887,2034.225830,2065.774170,1697.774170,-19.691665,-11.797437,B16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1633,6282,595,917,701,3269,2031,1089,-17.514410,-10.643085,607.419373,...,-11.388809,580.967712,900.387085,881.419373,2308.548340,2218.225830,1640.161255,-18.746541,-11.958173,B16
1634,6283,596,933,715,3298,2029,1081,-16.840671,-10.995155,616.516113,...,-10.469907,587.806458,905.129028,889.193542,2338.161377,2195.225830,1626.516113,-18.886008,-12.169379,B16
1635,6284,597,934,718,3298,2022,1080,-17.695548,-11.030213,616.645142,...,-8.711516,588.258057,905.483887,889.967712,2338.161377,2188.516113,1620.000000,-19.591463,-12.543976,B16
1636,6285,617,943,739,3437,2020,1102,-17.431293,-11.956398,601.967712,...,-9.991491,579.387085,906.483887,891.516113,2340.322510,2204.870850,1637.000000,-19.280141,-12.378779,B16


In [25]:
labeled = add_lucas_labels(v2, pd.read_csv('/scratch/bbug/ayang1/raw_data/lucas/lucas_2018/copernicus_filtered/lucas_2018_filtered.csv'))

Loading files
Creating dataset of size 56804


100%|██████████| 56804/56804 [00:14<00:00, 3850.88it/s]


In [28]:
labeled = labeled.loc[labeled['LABEL'] != 'NOT_CROP']

In [29]:
labeled

,system:index,0_B1_mean_post_20180101,0_B2_mean_post_20180101,0_B3_mean_post_20180101,0_B4_mean_post_20180101,0_B5_mean_post_20180101,0_B7_mean_post_20180101,0_VH_20180101,0_VV_20180101,10_B1_mean_post_20180411,...,9_B2_mean_post_20180401,9_B3_mean_post_20180401,9_B4_mean_post_20180401,9_B5_mean_post_20180401,9_B7_mean_post_20180401,9_VH_20180401,9_VV_20180401,POINT_ID,.geo,LABEL
334,0000000000000000b248_0,405.0,527.0,643.0,1200.0,1553.0,1101.0,-22.599060,-13.180478,549.1290,...,674.19354,790.25806,1540.2903,1899.6451,1421.2580,-21.372542,-9.627020,53302384,"{""type"":""MultiPoint"",""coordinates"":[]}",B31
335,0000000000000000b248_1,405.0,527.0,643.0,1200.0,1553.0,1101.0,-22.599060,-13.180478,549.1290,...,674.19354,790.25806,1540.2903,1899.6451,1421.2580,-21.537278,-11.356184,53302384,"{""type"":""MultiPoint"",""coordinates"":[]}",B31
336,0000000000000000b248_2,405.0,527.0,643.0,1200.0,1553.0,1101.0,-22.178021,-14.075480,549.1290,...,674.19354,790.25806,1540.2903,1899.6451,1421.2580,-21.967142,-12.348263,53302384,"{""type"":""MultiPoint"",""coordinates"":[]}",B31
337,0000000000000000b248_3,412.0,504.0,639.0,1165.0,1623.0,1083.0,-21.992862,-13.547367,543.4839,...,654.90320,763.35486,1472.9678,1846.8064,1422.2903,-21.881833,-12.687194,53302384,"{""type"":""MultiPoint"",""coordinates"":[]}",B31
338,0000000000000000b248_4,447.0,600.0,708.0,1374.0,2000.0,1183.0,-21.624669,-14.286716,562.6774,...,729.51610,829.16130,1760.4839,1976.7742,1436.8387,-21.245468,-7.614109,53302384,"{""type"":""MultiPoint"",""coordinates"":[]}",B31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56799,0000000000000000c2d6_66,626.0,936.0,728.0,3299.0,2041.0,1106.0,-18.207258,-11.396415,616.0968,...,901.93550,892.93550,2303.8708,2247.2258,1672.2258,-18.445190,-13.317497,58082458,"{""type"":""MultiPoint"",""coordinates"":[]}",B16
56800,0000000000000000c2d6_67,626.0,936.0,728.0,3299.0,2041.0,1106.0,-17.209820,-10.985422,616.0968,...,901.93550,892.93550,2303.8708,2247.2258,1672.2258,-19.271086,-12.846430,58082458,"{""type"":""MultiPoint"",""coordinates"":[]}",B16
56801,0000000000000000c2d6_68,614.0,951.0,714.0,3491.0,2032.0,1096.0,-17.737460,-11.488277,602.4516,...,914.19354,881.70966,2351.9678,2189.7097,1613.0000,-19.569058,-13.009642,58082458,"{""type"":""MultiPoint"",""coordinates"":[]}",B16
56802,0000000000000000c2d6_69,626.0,936.0,728.0,3299.0,2041.0,1106.0,-15.079310,-11.282294,616.0968,...,901.93550,892.93550,2303.8708,2247.2258,1672.2258,-21.525744,-12.995615,58082458,"{""type"":""MultiPoint"",""coordinates"":[]}",B16
